In [1]:
!pip install flask flask-ngrok transformers tensorflow pyngrok



In [2]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile

# Paths to zip files in Google Drive
suicide_model_zip = "/content/drive/MyDrive/bert_suicide_model.zip"
emotion_model_zip = "/content/drive/MyDrive/bert_emotion_model.zip"
tokenizer_zip = "/content/drive/MyDrive/tokenizer (1).zip"

# Unzip models and tokenizer
unzip_paths = {
    suicide_model_zip: "/content/bert_suicide_model",
    emotion_model_zip: "/content/bert_emotion_model",
    tokenizer_zip: "/content/tokenizer (1)"
}

for zip_path, extract_path in unzip_paths.items():
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

print("Models and tokenizer extracted successfully!")


Models and tokenizer extracted successfully!


In [4]:
import tensorflow as tf
from transformers import AutoTokenizer

# Load models
suicide_model = tf.saved_model.load("/content/bert_suicide_model")
emotion_model = tf.saved_model.load("/content/bert_emotion_model")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer (1)")

print(" Models and tokenizer loaded successfully!")


 Models and tokenizer loaded successfully!


In [5]:
!pip install flask-ngrok


In [6]:
!pip install pyngrok


In [7]:
!ngrok authtoken 2spbWZszlYSTSI8KbUfE16df8XI_2ddKZWvBBquWLvKhWjU6E


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

# Expose the app via ngrok
public_url = ngrok.connect(5000)  # 5000 Flask port
print(f"Public URL: {public_url}")

# Suicide risk and emotion detection function
def detect_suicide_and_emotion(text):
    suicide_label_mapping = {0: "non-suicide", 1: "suicide"}
    emotion_label_mapping = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
    risk_levels = {"sadness": "Medium Risk", "anger": "Medium Risk", "fear": "Medium Risk"}

    # Tokenize input text for suicide detection
    inputs_suicide = tokenizer([text], padding="max_length", truncation=True, max_length=128, return_tensors="tf")
    inputs_suicide = {key: tf.cast(value, tf.int64) for key, value in inputs_suicide.items()}

    # Run suicide detection model
    infer_suicide = suicide_model.signatures['serving_default']
    suicide_outputs = infer_suicide(input_ids=inputs_suicide['input_ids'],
                                    attention_mask=inputs_suicide['attention_mask'],
                                    token_type_ids=inputs_suicide['token_type_ids'])

    # Get prediction
    suicide_prediction = tf.argmax(suicide_outputs['output_1'], axis=1).numpy()[0]
    suicide_label = suicide_label_mapping[suicide_prediction]

    if suicide_label == "suicide":
        return {"text": text, "suicide_risk": "High Risk"}

    # Tokenize for emotion detection
    inputs_emotion = tokenizer([text], padding="max_length", truncation=True, max_length=87, return_tensors="tf")
    inputs_emotion = {key: tf.cast(value, tf.int64) for key, value in inputs_emotion.items()}

    # Run emotion detection model
    infer_emotion = emotion_model.signatures['serving_default']
    emotion_outputs = infer_emotion(input_ids=inputs_emotion['input_ids'],
                                    attention_mask=inputs_emotion['attention_mask'],
                                    token_type_ids=inputs_emotion['token_type_ids'])

    # Get emotion prediction
    emotion_prediction = tf.argmax(emotion_outputs['output_1'], axis=1).numpy()[0]
    emotion_label = emotion_label_mapping[emotion_prediction]

    # Determine risk level
    risk_level = risk_levels.get(emotion_label, "Low Risk")

    return {"text": text, "emotion": emotion_label, "suicide_risk": risk_level}

# Flask API Route
@app.route("/analyze", methods=["POST"])
def analyze():
    data = request.json
    text = data.get("text", "")

    if not text:
        return jsonify({"error": "Text input is required!"}), 400

    result = detect_suicide_and_emotion(text)
    return jsonify(result)

if __name__ == "__main__":
    app.run()



Public URL: NgrokTunnel: "https://fdd2-34-19-45-84.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:19:21] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:19:38] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:19:51] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:20:07] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:20:25] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:20:42] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:20:49] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:20:59] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:21:26] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Feb/2025 04:22:10] "POST /